In [ ]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import http.client
import urllib.parse
import snowflake.connector
from sqlalchemy import create_engine
from joblib import Parallel, delayed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import gspread
from google.oauth2.service_account import Credentials
from io import StringIO

In [2]:
load_dotenv()
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')
SERVICE_ACCOUNT_FILE = os.getenv('SERVICE_ACCOUNT_FILE')

# Rate limit control
rate_lock = threading.Lock()
last_request_time = 0


#Gsheet credential
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]
creds = Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=SCOPES)

gc = gspread.authorize(creds)

## Connect to  Snowflake database for raw data initial processing

In [ ]:
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

In [ ]:
spreadsheet = gc.open_by_key("1I-Q-nFlp--jLXYSrS8TuOvxZO62KvhTdDSn1iLDBTkQ")
worksheets = spreadsheet.worksheets()

gspread.worksheet.Worksheet

### 1. Read in DA Job Analysis

In [5]:
def get_job_descriptions():
    spreadsheet = gc.open_by_key("1I-Q-nFlp--jLXYSrS8TuOvxZO62KvhTdDSn1iLDBTkQ")
    worksheets = spreadsheet.worksheets()

    df_jobs = pd.DataFrame()
    for worksheet in worksheets[1:]: #omit the first sheet/master sheet
        print(worksheet)
        all_data = worksheet.get_all_values()
        headers = all_data[0]
        rows = all_data[1:]
        df_data = pd.DataFrame(rows, columns=headers)
        df_data = df_data.drop_duplicates(subset=['Job_Description'], keep='first')
        df_jobs = pd.concat([df_data, df_jobs],ignore_index=True)
    return df_jobs

df_jobs = get_job_descriptions()
df_jobs.shape
df_jobs.head()

<Worksheet 'da_description' id:819730630>
<Worksheet 'de_description' id:1530019367>
<Worksheet 'ds_description' id:244877389>


,Job_Title,Job_Description
0,Data Scientist,About the job\r\nJoin an exciting technology s...
1,Data Scientist,About the job\r\nYum! Brands is the world’s la...
2,Data Scientist,About the job\r\nJoin Vertical Scope Group’s (...
3,Data Scientist,Job Description\r\n\r\nThe Operations Research...
4,Data Scientist,About the job\r\nJoin Australia's leading onli...


In [ ]:
def single_job_desciption_list(df_jobs):
    job_dict={}

    df_da = df_jobs[df_jobs['Job_Title'] == 'Data Analyst']
    df_ds = df_jobs[df_jobs['Job_Title'] == 'Data Scientist']
    df_de = df_jobs[df_jobs['Job_Title'] == 'Data Engineer']

    da_joined_text = '\n'.join(df_da['Job_Description'].unique().tolist())
    job_dict['Data Analyst'] = [da_joined_text]


    ds_joined_text = '\n'.join(df_ds['Job_Description'].unique().tolist())
    job_dict['Data Scientist'] = [ds_joined_text]

    de_joined_text = '\n'.join(df_de['Job_Description'].unique().tolist())
    job_dict['Data Engineer'] = [de_joined_text]

    return job_dict

job_dict = single_job_desciption_list(df_jobs)
job_dict

{'Data Analyst': ["The Suburban Connect consortium, comprising CPB Contractors, ACCIONA and Ghella, have been selected by the Victorian Government to deliver the first major tunnelling package on the Suburban Rail Loop (SRL) East project.The package includes construction of a 16-kilometre section of the project’s twin tunnels, including tunnelling between Cheltenham and Glen Waverley, 55 safety cross passages between the tunnels, station boxes at Clayton and Monash and the construction of portals – the entrances and exits to the tunnels – at the stabling facility.About The OpportunityJoin our team as a Health & Safety Data Analyst and you will be pivotal in helping us to understand our health and safety risk profile and finding new and innovative ways to achieve an industry leading health and safety performance.The Key Responsibilities Of The Role Include      Sourcing health and safety data from various inputs and maintaining the health and safety data lake.Creating dynamic dashboards

In [ ]:
def call_deepseek_api(job_dict):
    # combine all descriptions of a job title to a single list
    all_descriptions = []
    for job_title, descriptions in job_dict.items():
        for desc in descriptions:
            all_descriptions.append(f"{job_title}: {desc}")

    prompt = f"""
Here are various job descriptions for multiple data-related roles (job title is before each colon):

{chr(10).join(all_descriptions)}

Please analyze all above descriptions and GROUP them into only three categories based on their role:
1. Data Analyst (all related titles)
2. Data Scientist (all related titles)
3. Data Engineer (all related titles)

For each category, list Top 5 Technical Skills and Top 5 Soft Skills (based on frequency).

Return a CSV with columns:
job_title, Top 5 Technical Skills, Top 5 Soft Skills

No explanation, CSV ONLY.
"""

    url = 'https://api.deepseek.com/v1/chat/completions'
    headers = {
        "Authorization": f"Bearer {os.getenv('DEEPSEEK_API_KEY')}",
        "Content-Type": "application/json",
        "User-Agent": "JobClassification/1.0 (Python)"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    answer = response.json()['choices'][0]['message']['content']

    #only keep the first csv section, drop all other explanation, markdown and notations etc.
    lines = answer.strip().splitlines()
    header_idx = None
    for idx, line in enumerate(lines):
        if line.lower().startswith("job_title"):
            header_idx = idx
            break
    if header_idx is not None:
        csv_text = '\n'.join(lines[header_idx:header_idx+4])
    else:
        csv_text = answer.strip()


    df = pd.read_csv(StringIO(csv_text))
    return df


df_job_analysis = call_deepseek_api(job_dict)
print(df_job_analysis)

        job_title                             Top 5 Technical Skills  \
0    Data Analyst              SQL, Power BI, Excel, Python, Tableau   
1  Data Scientist  Python, Machine Learning, SQL, R, TensorFlow/P...   
2   Data Engineer                     SQL, Python, AWS, Azure, Spark   

                                   Top 5 Soft Skills  
0  Communication, Analytical Thinking, Problem-So...  
1  Problem-Solving, Communication, Analytical Thi...  
2  Problem-Solving, Collaboration, Communication,...  


In [41]:
df_job_analysis

,job_title,Top 5 Technical Skills,Top 5 Soft Skills
0,Data Analyst,"SQL, Power BI, Excel, Python, Tableau","Communication, Analytical Thinking, Problem-So..."
1,Data Scientist,"Python, Machine Learning, SQL, R, TensorFlow/P...","Problem-Solving, Communication, Analytical Thi..."
2,Data Engineer,"SQL, Python, AWS, Azure, Spark","Problem-Solving, Collaboration, Communication,..."


In [43]:
#write to a new snowflake table for seniority

def load_to_snowflake(df_job_analysis):
    # Create a Snowflake connection engine
   engine = create_engine(
        'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
        user="NIKKILW2025",
        password=snowflake_password,
        account="gbszkwp-by30611",
        warehouse="SNOWFLAKE_LEARNING_WH",
        database="linkedin_db",
        schema="linkedin_raw"
    )
   )

   table_name = "mart_top_job_skills"

   df_job_analysis.to_sql(
        name=table_name,
        con=engine,
        if_exists='replace',
        index=False
    )

   print(f"Data loaded to Snowflake table {table_name} successfully.")


load_to_snowflake(df_job_analysis)

Data loaded to Snowflake table mart_top_job_skills successfully.
